In [2]:
import pandas as pd
import numpy as np

In [30]:
PARAMETERS = {
    'nf': 2, # noise figure
    'ci': 9, # carrier to inference ratio
    'tx_macro_gain': 17,
    'tx_micro_gain': 5,
    'tx_macro_losses': 2,
    'tx_micro_losses': 1,
    'grxd': 3, #  polarization diversity of the antenna value for urban environment
    'gfh': 2, # gain produced by the use of frequency hopping 
    'mf': 4.2, # shadow-fading margin
    ## UMTS
    'ebn': 5.6, # e energy per bit per noise ratio and is 5.6 dB for CS voice
    'gpr': 25, # processing gain
    'gsho': 3, # soft handover gain
    'mi': 3, #  interference margin -- for 50% load
    'ms': 3 # saturation margin
}

In [7]:
def watt_dbm(p):
    return 10*np.log(1000*p)


def dbm_watt(p):
    return (10**(p/10))/1000

In [4]:
# ktb = 10*np.log(K*T) 
# where K = Boltzmann constant = 1.380649 × 10^-23 m2 kg s-2 K-1
# T = temperature in Kelvin = 293K
ktb = -599.595

In [26]:
def power_calc(radio_type, bw): 
    '''
    Parameters
    ----------
    radio_type: string
        Type of radio tower being used
    Returns
    -------
    power: float 
        Transmitter power in dBm
    '''  
    if radio_type == 'GSM':
        power_gsm =   \
                    10*np.log(bw) + \
                    PARAMETERS['nf'] + \
                    PARAMETERS['ci'] - \
                    PARAMETERS['tx_macro_gain'] + \
                    PARAMETERS['tx_macro_losses'] - \
                    PARAMETERS['grxd'] - \
                    PARAMETERS['gfh'] + \
                    PARAMETERS['mf']    
        return power_gsm
    else:
        power_umts = \
                10*np.log(bw) + \
                PARAMETERS['nf'] + \
                PARAMETERS['ebn'] - \
                PARAMETERS['tx_macro_gain'] + \
                PARAMETERS['tx_macro_losses'] - \
                PARAMETERS['grxd'] - \
                PARAMETERS['gpr'] - \
                PARAMETERS['gsho'] + \
                PARAMETERS['mi'] + \
                PARAMETERS['ms'] + \
                PARAMETERS['mf']
        return power_umts


In [31]:
radio_type = 'GSM'
power_calc(radio_type, 200)

48.18317366548037

In [32]:
radio_type = 'UMTS'
power_calc(radio_type, 3840)

52.33227645581772

In [ ]:
def power_util(p, ant_type):
    '''
    Calculates the Effective Radiated Power (ERP) using the formula: 
    ERP = tx_power * antenna_gain * feedline_loss

    Parameters
    ---------- 
    df: pandas dataframe
        dataframe generated by heuristic calculations
    params: dict
        antenna (transmitter and receiver) gains and losses

    Returns
    -------
    erp: int
        Effective Radiated Power
    '''
    
    if ant_type == 'macro':
       erp = watt_dbm(p) + PARAMETERS['tx_macro_gain'] - PARAMETERS['tx_macro_losses'], df['power']))
    else:
        erp = watt_dbm(p) + PARAMETERS['tx_micro_gain'] - PARAMETERS['tx_micro_losses'], df['power']))
    return dbm_watt(erp)

In [ ]:
def energy_util(p, n, t, mcs_index=0):
    '''
    Calculates the effective energy consumed by the specific service

    Parameters
    ----------
    p: int
        transmit power per prb pair
    n: int
        number of prb pairs
    t: int
        duration of transmit (in hours)

    is the power offset of each optimal MCS to the basic MCS

    Returns
    -------
    energy: energy utilization : int (units = Watt-hour)
    '''
    # mcs_delta = mcs_i
    p_tot = p  # * mcs_delta * n
    energy = p_tot * n * t
    return energy

In [ ]:
def cost(energy, usd):
    return energy * usd

In [ ]:
def driver():
    print("Transmit Power estimation for GSM: ", power_calc("GSM"))
    print("ERP estimation: ", )
    print("Total Energy estimation: ")
    print("Total cost estimation: ")